In [5]:
import pandas as pd
import openpyxl
import inspect
from openpyxl.worksheet.worksheet import Worksheet
from openpyxl.utils.dataframe import dataframe_to_rows

In [6]:
# Create mock dataframes
df1 = pd.DataFrame({'A': [1, 2, 3], 'B': ['a', 'b', 'c'], 'C': [True, False, True]})
df2 = pd.DataFrame({'X': ['a', 'b', 'c', 'd'], 'Y': [1, 2, 3, 4], 'Z': [True, False, True, False]})

In [8]:
# function that receives a dataframe as a parameter and returns its name as a string
def get_df_name(df):
    caller_globals = inspect.currentframe().f_back.f_globals
    
    for name, obj in caller_globals.items():
        if isinstance(obj, pd.DataFrame) and obj is df:
            return name
    return None

df_name = get_df_name(df1)
df_name

'df1'

In [23]:
# function that receives a dataframe as a parameter and returns its name as a string

def get_df_name(df):
    caller_globals = inspect.currentframe().f_back.f_globals
    
    for name, obj in caller_globals.items():
        if isinstance(obj, pd.DataFrame) and obj is df:
            return name
    return None

def export_to_excel(dataframes, filename):
    # Create a workbook
    workbook = openpyxl.Workbook()

    for df in dataframes:

        # get the name of the dataframe
        sheet_name = get_df_name(df)

        # Create a worksheet
        worksheet = workbook.create_sheet(sheet_name)
        
        # Write the data from the dataframe to the worksheet
        for row in dataframe_to_rows(df, index=False, header=True):
            worksheet.append(row)
        
        # Set the column widths
        for column in df:
            column_length = max(df[column].astype(str).map(len).max(), len(column)) + 10
            col_idx = df.columns.get_loc(column)
            worksheet.column_dimensions[openpyxl.utils.get_column_letter(col_idx+1)].width = column_length

        # Freeze the header row
        worksheet.freeze_panes = worksheet['A2']
        
        # Enable filters for the worksheet
        num_rows, num_cols = df.shape
        cell_range = f'A1:{openpyxl.utils.get_column_letter(num_cols)}{num_rows}'
        worksheet.auto_filter.ref = cell_range
    
    del workbook['Sheet']
    
    # Save the workbook
    workbook.save(filename)

In [21]:
# Call the export_to_excel function
export_to_excel([df1, df2], ['df1','df2'],'./files/excelwriter.xlsx')